In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import yfinance as yf

tickers = pd.read_csv('nasdaq_screener.csv')['Symbol'].values

predictions = {}
for i, ticker in enumerate(tickers):
    try:
        market = yf.Ticker(ticker).history(period='max')
        market.index = pd.to_datetime(market.index)
        market = market.drop(["Dividends", "Stock Splits"], axis=1)

        horizons = [2, 8, 32, 128, 512]
        predictors = []
        for horizon in horizons:
            rolling_averages = market.rolling(horizon).mean()
            ratio_column = f"Close_Ratio_{horizon}"
            market[ratio_column] = market["Close"] / rolling_averages["Close"]
            trend_column = f"Trend_{horizon}"
            market[trend_column] = market.shift(1).rolling(horizon).sum()["Close"].pct_change()
            predictors += [ratio_column, trend_column]

        market = market.dropna()

        market["Target"] = market["Close"].shift(-5) > market["Close"]

        model = RandomForestClassifier(n_estimators=250, min_samples_split=50, random_state=0)
        model.fit(market[predictors][:-5], market["Target"][:-5])

        last_data = market[predictors].iloc[-1:]
        prediction = model.predict_proba(last_data)[0, 1]

        if i % 100 == 0:
            print(f"{i+1} out of {len(tickers)} complete")
        predictions[ticker] = prediction

    except Exception as error:
        continue

sorted_preds = sorted(predictions.items(), key=lambda x:x[1])
sorted_preds[-10:]